In the context of the moving average crossover strategy, the ‘signal’ column in the signals DataFrame represents the trading signals generated by the strategy. Here’s what the values mean:

1: This represents a “Buy” signal. It indicates that the short-term moving average has crossed above the long-term moving average, suggesting an upward trend in the stock price. This is typically considered a bullish signal, and traders might consider entering a long position or buying the stock.

0: This represents a “Sell” signal. It indicates that the short-term moving average has crossed below the long-term moving average, suggesting a downward trend in the stock price. This is typically considered a bearish signal, and traders might consider exiting a long position or selling the stock.

In [1]:

#%pip install tqdm
# Import necessary libraries
import itertools
# Import necessary libraries
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from tqdm import tqdm
from multiprocessing import Pool, cpu_count


# Function to calculate RSI
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()        
    up_chg = 0 * diff
    down_chg = 0 * diff
    up_chg[diff > 0] = diff[ diff>0 ]
    down_chg[diff < 0] = diff[ diff < 0 ]
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

today = dt.date.today()

#####################################################
# Download historical data for desired ticker symbol
ticker = "TSLA"

#####################################################


tickerData = yf.Ticker(ticker)
df = tickerData.history(period='1d', start='2021-1-1', end=today,auto_adjust=False)

####################################################



In [2]:

# Define the range of possible values for short_window and long_window
short_window_range = range(1, 61)
long_window_range = range(1, 261)

# Initialize variables to store the optimal values
optimal_short_window = None
optimal_long_window = None
max_portfolio_value = -np.inf

# Calculate the total number of iterations
#total_iterations = len(short_window_range) * len(long_window_range)
total_iterations = 13770

# Create a progress bar
progress_bar = tqdm(total=total_iterations, desc='Optimizing', bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')

# Iterate over all combinations of short_window and long_window
for short_window, long_window in itertools.product(short_window_range, long_window_range):
    # Skip if short_window is not less than long_window
    if short_window >= long_window:
        continue

    # Calculate the moving averages
    signals = pd.DataFrame(index=df.index)
    signals['signal'] = 0.0
    signals['short_mavg'] = df['Close'].rolling(window=short_window, min_periods=1, center=False).mean()
    signals['long_mavg'] = df['Close'].rolling(window=long_window, min_periods=1, center=False).mean()

    # Create signals
    signals['signal'][short_window:] = np.where(signals['short_mavg'][short_window:] 
                                                > signals['long_mavg'][short_window:], 1.0, 0.0)   
    signals['positions'] = signals['signal'].diff()

    # Initialize the portfolio with the initial investment
    portfolio = 10000
    # Initialize a variable to hold the number of shares
    shares = 0

    # Iterate over the signals DataFrame
    for date, row in signals.iterrows():
        # Get the close price for this date
        close_price = df.loc[date, 'Close']

        # If the signal is 1.0 (buy) and we have enough money to buy at least one share, buy as many shares as we can
        if row['signal'] == 1.0 and portfolio >= close_price:
            shares_bought = portfolio // close_price
            portfolio -= shares_bought * close_price
            shares += shares_bought
        # If the signal is 0.0 (sell), sell all our shares
        elif row['signal'] == 0.0 and shares > 0:
            portfolio += shares * close_price
            shares = 0

    # Calculate the final value of the portfolio (cash + value of shares)
    final_portfolio_value = portfolio + shares * close_price

    # Update the optimal values if the current final portfolio value is greater than the maximum final portfolio value found so far
    if final_portfolio_value > max_portfolio_value:
        max_portfolio_value = final_portfolio_value
        optimal_short_window = short_window
        optimal_long_window = long_window

    # Update the progress bar
    progress_bar.update()

# Close the progress bar
#progress_bar.close()

print(f"The optimal period for short_window is {optimal_short_window}")
print(f"The optimal period for long_window is {optimal_long_window}")
print(f"The highest final portfolio value is {max_portfolio_value}")


Optimizing: 100%|██████████| 13770/13770

The optimal period for short_window is 3
The optimal period for long_window is 25
The highest final portfolio value is 30865.20574951172


Optimizing: 100%|██████████| 13770/13770


# Define the range of possible values for short_window and long_window
short_window_range = range(1, 61)
long_window_range = range(1, 261)

# Initialize variables to store the optimal values
optimal_short_window = None
optimal_long_window = None
max_portfolio_value = -np.inf

# Calculate the total number of iterations
total_iterations = len(short_window_range) * len(long_window_range)

# Define a function to calculate the final portfolio value for a given combination of short_window and long_window
def calculate_portfolio_value(params):
    short_window, long_window = params

    # Calculate the moving averages
    signals['short_mavg'] = df['Close'].rolling(window=short_window, min_periods=1, center=False).mean()
    signals['long_mavg'] = df['Close'].rolling(window=long_window, min_periods=1, center=False).mean()

    # Create signals
    signals['signal'][short_window:] = np.where(signals['short_mavg'][short_window:] 
                                                > signals['long_mavg'][short_window:], 1.0, 0.0)   
    signals['positions'] = signals['signal'].diff()

    # Initialize the portfolio with the initial investment
    portfolio = 10000
    # Initialize a variable to hold the number of shares
    shares = 0

    # Iterate over the signals DataFrame
    for date, row in signals.iterrows():
        # Get the close price for this date
        close_price = df.loc[date, 'Close']

        # If the signal is 1.0 (buy) and we have enough money to buy at least one share, buy as many shares as we can
        if row['signal'] == 1.0 and portfolio >= close_price:
            shares_bought = portfolio // close_price
            portfolio -= shares_bought * close_price
            shares += shares_bought
        # If the signal is 0.0 (sell), sell all our shares
        elif row['signal'] == 0.0 and shares > 0:
            portfolio += shares * close_price
            shares = 0

    # Calculate the final value of the portfolio (cash + value of shares)
    final_portfolio_value = portfolio + shares * close_price

    return short_window, long_window, final_portfolio_value

# Create a list of all combinations of short_window and long_window
params = [(short_window, long_window) for short_window in short_window_range for long_window in long_window_range if short_window < long_window]

# Create a multiprocessing Pool
with Pool(processes=cpu_count()) as pool:
    # Use the Pool's imap function to calculate the final portfolio value for each combination of short_window and long_window
    results = []
    with tqdm(total=total_iterations, desc='Optimizing', bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}') as pbar:
        for short_window, long_window, final_portfolio_value in pool.imap(calculate_portfolio_value, params):
            results.append((short_window, long_window, final_portfolio_value))
    pbar.update()

# Find the combination of short_window and long_window that gives the highest final portfolio value
optimal_short_window, optimal_long_window, max_portfolio_value = max(results, key=lambda x: x[2])

print(f"The optimal period for short_window is {optimal_short_window}")
print(f"The optimal period for long_window is {optimal_long_window}")
print(f"The highest final portfolio value is {max_portfolio_value}")